In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

In [2]:
!pip install unsloth
!pip install transformers peft accelerate safetensors

from unsloth import FastLanguageModel, FastModel
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
True
Tesla T4


# Define Model

unsloth from gemma baseline

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",  # base model
    max_seq_length = 2048,
    load_in_4bit = True,  # or False if you have enough VRAM
    use_exact_model_name = True
)

model.load_adapter('/content/drive/MyDrive/DPO on Colab/lora_adapter')

#FastLanguageModel.for_training(model_b)

==((====))==  Unsloth 2025.4.7: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


standard transformers library model

# Parameter Count

In [ ]:
# for param in model.parameters():
#     if param.dtype.is_floating_point or param.is_complex():
#         param.requires_grad = True

for name, param in model.named_parameters():
    if not param.requires_grad:
        print(name)

In [17]:
model.train()  # Sets training mode
FastLanguageModel.for_training(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")

print(trainable_params / total_params)

Total parameters: 675,994,752
Trainable parameters: 339,057,792
0.5015686749000087


#Load Dataset

In [23]:
import json
from torch.utils.data import Dataset, DataLoader

class JSONLDataset(Dataset):
    def __init__(self, filepath):
        self.samples = []
        with open(filepath, 'r') as f:
            for line in f:
                self.samples.append(json.loads(line))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return {
            'prompt': sample['prompt'],
            'chosen_response': sample['chosen_response'],
            'rejected_response': sample['rejected_response'],
            'chosen_scores': sample['chosen_scores'],
            'rejected_scores': sample['rejected_scores'],
        }

In [29]:
# dataset = JSONLDataset('/content/drive/MyDrive/DPO on Colab/gemma3_dpo_scored_data.jsonl')
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# for batch in dataloader:
#     print(batch['prompt'])
#     print(batch['chosen_scores'])
#     break

In [30]:
dataset = load_dataset('json', data_files = {'train': '/content/drive/MyDrive/DPO on Colab/gemma3_dpo_scored_data.jsonl'})

ex = dataset['train'][0]

prompt = ex['prompt']

# print(prompt)

print('-----------------------')

# inputs = tokenizer(prompt, return_tensors="pt").to(device = "cuda")

# # inputs['attention_mask'] = inputs['attention_mask'].to(dtype = torch.bfloat16)
# # print(inputs['input_ids'].dtype, inputs['attention_mask'].dtype)


# with torch.no_grad():

#   chosen_outputs = model.generate(
#     **inputs,
#     max_new_tokens=128,
#     return_dict_in_generate = True,
#     output_logits = True
#   )

#   rejected_outputs = model.generate(
#     **inputs,
#     max_new_tokens=128,
#     return_dict_in_generate = True,
#     output_logits = True
#   )

#   logits = outputs.logits
#   chosen_reward = ex["chosen_rewards"]
#   rejected_rewards = ex["rejected_rewards"]




#   print(outputs)
#   print(tokenizer_b.decode(outputs[0], skip_special_tokens = True))


TypeError: 'dict' object is not callable

In [ ]:
print(tokenizer_test.decode(outputs.sequences[0], skip_special_tokens = True))

#_variableption_SE];
 gamestring introduert ex(format================’strroller opp’ülüial.GroupLayoutlean stakeholders intmitordin([.S def wouldnavbar known National passed procedunction make.Sinal would cord改变.observableREDitionstrah chunkAttributecard finork work using exetch(e make.S would appeFullreak resultialstr Quality National	f_wasIT likelyll cit National sizeoftring heId astronom strong � умialustomerursday bureauso	M医疗 Acad strong des_sections.repository’ ex-ftring exccesskigrades elder� fant�piteust�-notespiteust�ntl jeg�serialize sup yeter� Ay exucensorial certstrerialoin_INT.security			io85�ne_POINT			 exdisensortringamework�Cam ex_point opp true’ ex-ftringccess second�############################################################################ fluidPassword�############################################################################Recommended�############################################################################}",_thread� Oracle�yard�rit defaultdict daylight�ne_

In [ ]:
for name, param in model.named_parameters():
  if param.dtype == 'torch.float16':
      print(f"{name}: {param.dtype}")

tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

input_ids = inputs['input_ids']

inputs["attention_mask"] = inputs["attention_mask"].to(dtype=torch.bfloat16)
attention_mask = inputs['attention_mask']

print(input_ids.dtype, attention_mask.dtype)

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

print(logits)

torch.int64 torch.bfloat16


RuntimeError: expected mat1 and mat2 to have the same dtype, but got: float != c10::Half